In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#Lecture des fichiers
trainInitial = pd.read_csv("file:///Users/lionel.capo-chichi/Desktop/Eolienne/input_training.csv",sep=';')
test = pd.read_csv("file:///Users/lionel.capo-chichi/Desktop/Eolienne/input_testing.csv",sep=';')
output = pd.read_csv("file:///Users/lionel.capo-chichi/Desktop/Eolienne/45.csv",sep=';')

In [4]:
train = pd.merge(trainInitial,output,on='ID')

In [5]:
train['MAC_CODE'] = train['MAC_CODE'].replace('WT3',3)
train['MAC_CODE'] = train['MAC_CODE'].replace('WT1',1)
train['MAC_CODE'] = train['MAC_CODE'].replace('WT2',2)
train['MAC_CODE'] = train['MAC_CODE'].replace('WT4',4)
test['MAC_CODE'] = test['MAC_CODE'].replace('WT3',3)
test['MAC_CODE'] = test['MAC_CODE'].replace('WT1',1)
test['MAC_CODE'] = test['MAC_CODE'].replace('WT2',2)
test['MAC_CODE'] = test['MAC_CODE'].replace('WT4',4)

In [6]:
train.columns

Index(['ID', 'MAC_CODE', 'Date_time', 'Pitch_angle', 'Pitch_angle_min',
       'Pitch_angle_max', 'Pitch_angle_std', 'Hub_temperature',
       'Hub_temperature_min', 'Hub_temperature_max', 'Hub_temperature_std',
       'Generator_converter_speed', 'Generator_converter_speed_min',
       'Generator_converter_speed_max', 'Generator_converter_speed_std',
       'Generator_speed', 'Generator_speed_min', 'Generator_speed_max',
       'Generator_speed_std', 'Generator_bearing_1_temperature',
       'Generator_bearing_1_temperature_min',
       'Generator_bearing_1_temperature_max',
       'Generator_bearing_1_temperature_std',
       'Generator_bearing_2_temperature',
       'Generator_bearing_2_temperature_min',
       'Generator_bearing_2_temperature_max',
       'Generator_bearing_2_temperature_std', 'Generator_stator_temperature',
       'Generator_stator_temperature_min', 'Generator_stator_temperature_max',
       'Generator_stator_temperature_std', 'Gearbox_bearing_1_temperature',
    

In [7]:
y = train['TARGET']

In [8]:
train = train.drop(['TARGET','Date_time'],axis=1)
data = pd.concat([train,test],axis=0)


In [9]:
listColumns = []
for v in data.columns:
    if v not in ['ID','MAC_CODE']:
        listColumns.append(v)
        
data1 = data.copy()
data1['ID']  = data1['ID'] -1

for col in listColumns :
    data1.rename(columns={col: col +('_lag_1')},inplace=True)
dataMerge1 =pd.merge(data,data1,on=['ID','MAC_CODE'],how='left') 

data2 = data.copy()
data2['ID']  = data2['ID'] -2

for col in listColumns :
    data2.rename(columns={col: col +('_lag_2')},inplace=True)
dataMerge2 =pd.merge(dataMerge1,data2,on=['ID','MAC_CODE'],how='left') 

In [10]:
dataMerge2 = dataMerge2.fillna(dataMerge2.mean())


In [11]:
dataMerge2['Jour'] = dataMerge2['ID']%(6*24)


In [12]:
X_train = dataMerge2.iloc[:308693]
X_val = dataMerge2.iloc[308693:463039]
X_test_ctx = dataMerge2.iloc[463039:617386]
y_train = y[:308693]
y_val = y[308693:463039]
y_test_ctx = y[463039:617386]

In [13]:
from lightgbm import LGBMRegressor
lgb = LGBMRegressor()

lgb.fit(X_train, y_train)
predictionslgb = lgb.predict(X_val)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_val, predictionslgb)
#Mieux avec la semaine et l'olympiade

12.859720172802591

In [15]:
predictionslgb

array([905.07921522, 782.55932145, 985.0796851 , ...,  -2.19627266,
        -2.19627266,  -2.19627266])

In [25]:
from sklearn import linear_model
lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)
predictionslr = lr.predict(X_val)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_val, predictionslr)
#Mieux avec la semaine et l'olympiade

91.58995740708693

In [26]:
test_preds_lgb = lgb.predict(X_test_ctx)
test_preds_lr = lr.predict(X_test_ctx)

In [27]:
stacked_pred = np.column_stack((predictionslgb,predictionslr))
stacked_test_pred = np.column_stack((test_preds_lgb,test_preds_lr))

In [28]:
meta_model = linear_model.LinearRegression()
meta_model.fit(stacked_pred,y_val)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [29]:
final_predictions = meta_model.predict(stacked_test_pred)

In [30]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test_ctx, final_predictions)

17.14028742171079

In [31]:
X_train = dataMerge2.iloc[:463039]
X_val = dataMerge2.iloc[463039:617386]
X_test = dataMerge2.iloc[617386:]
y_train = y[:463039]
y_val = y[463039:617386]
y_test = y[617386:]

In [32]:
from lightgbm import LGBMRegressor
lgb = LGBMRegressor()

lgb.fit(X_train, y_train)
predictionslgb = lgb.predict(X_val)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_val, predictionslgb)
#Mieux avec la semaine et l'olympiade

17.148315855853397

In [33]:
from sklearn import linear_model
lr = linear_model.LinearRegression()
lr.fit(X_train, y_train)
predictionslr = lr.predict(X_val)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_val, predictionslr)
#Mieux avec la semaine et l'olympiade

104.83528837241549

In [36]:
test_preds_lgb = lgb.predict(X_test)
test_preds_lr = lr.predict(X_test)
stacked_pred = np.column_stack((predictionslgb,predictionslr))
stacked_test_pred = np.column_stack((test_preds_lgb,test_preds_lr))
meta_model = linear_model.LinearRegression()
meta_model.fit(stacked_pred,y_val)
final_predictions = meta_model.predict(stacked_test_pred)

In [37]:
#test = test.reset_index()
sub_df = pd.DataFrame()
sub_df['ID'] = test['ID']
sub_df["TARGET"] = final_predictions
sub_df.to_csv("stackinglgblr.csv", index=False)

In [ ]:
dataMerge10.columns

In [ ]:
X_train.shape

In [ ]:
X_val.shape

In [ ]:
from lightgbm import LGBMRegressor
lgb = LGBMRegressor()

lgb.fit(X_train, y_train)
predictionslgb = lgb.predict(X_val)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_val, predictionslgb)
#Mieux avec la semaine et l'olympiade

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.regularizers import l1, l2
import tensorflow as tf
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from keras import optimizers
from keras import regularizers

BATCH_SIZE = 256
EPOCHS = 60
LEARNING_RATE = 0.0001
DATASET_SIZE = 7000000

model = Sequential()
model.add(Dense(256, activation='relu', input_dim=X_train.shape[1]))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1))

adam = optimizers.adam(lr=LEARNING_RATE)
model.compile(loss='mse', optimizer=adam, metrics=['mae'])

In [ ]:
model.summary()

hist = model.fit(X_train, y_train, validation_data = (X_val, y_val))


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 231))
# Adding the second hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))
# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

# Compiling Neural Network
classifier.compile(optimizer = 'adam', loss = 'mae', metrics = ['mae'])

# Fitting our model 
classifier.fit(X_train, y_train, batch_size = 10, nb_epoch = 100)
y_pred = classifier.predict(X_test)


In [ ]:
#test = test.reset_index()
sub_df = pd.DataFrame()
sub_df['ID'] = test['ID']
sub_df["TARGET"] = lgb.predict(X_test)
sub_df.to_csv("randomForest140.csv", index=False)

In [ ]:

from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor((10, 1), early_stopping=False)

mlp.fit(X_train_std, y_train)
predictionsmlp = mlp.predict(X_val_std)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_val, predictionsmlp)
#Mieux avec la semaine et l'olympiade



In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X_train, y_train)
predictionsknn = knn.predict(X_val)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_val, predictionsknn)
#Mieux avec la semaine et l'olympiade



In [ ]:
from sklearn.svm import SVR
svr = SVR()

svr.fit(X_train, y_train)
predictionssvr = svr.predict(X_val)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_val, predictionssvr)
#Mieux avec la semaine et l'olympiade                                                                                                                                                                                                                                                                                                                                             

In [ ]:
#test = test.reset_index()
sub_df = pd.DataFrame()
sub_df['ID'] = test['ID']
sub_df["TARGET"] = lgb.predict(X_test_std)
sub_df.to_csv("randomForest135.csv", index=False)

In [ ]:
from sklearn.feature_selection import RFECV



rfecv = RFECV( estimator = lgb , step = 1 , scoring = 'neg_mean_absolute_error' )
rfecv.fit( X_train , y_train )


print (rfecv.score( X_train , y_train ) , rfecv.score( X_val , model.predict(X_val) ))
print( "Optimal number of features : %d" % rfecv.n_features_ )
predictions = model.predict(X_val)



In [ ]:
#test = test.reset_index()
sub_df = pd.DataFrame()
sub_df['ID'] = test['ID']
sub_df["TARGET"] = 0.637* lgb.predict(X_test) + (1-0.637)*ad.predict(X_test)
sub_df.to_csv("randomForest125.csv", index=False)

In [ ]:
feature_importances = pd.DataFrame(lgb.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',                                                               
                                                                        ascending=False)
print(feature_importances)

In [ ]:
plt.boxplot(X_train['Generator_speed'])
plt.show()

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
ad = AdaBoostRegressor(DecisionTreeRegressor())

ad.fit(X_train, y_train)
predictionsAd = ad.predict(X_val)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_val, predictionsAd)
#Mieux avec la semaine et l'olympiade

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor()

xgb.fit(X_train, y_train)
predictionsXgb = xgb.predict(X_val)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_val, predictionsXgb)
#Mieux avec la semaine et l'olympiade

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

rf.fit(X_train, y_train)
predictionsRf = rf.predict(X_val)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_val, predictionsRf)
#Mieux avec la semaine et l'olympiade

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
et = ExtraTreesRegressor()

et.fit(X_train, y_train)
predictionsEt = rf.predict(X_val)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_val, predictionsEt)
#Mieux avec la semaine et l'olympiade

In [ ]:
feature_importances = pd.DataFrame(xgb.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',                                                               
                                                                        ascending=False)
print(feature_importances)

In [ ]:
#test = test.reset_index()
sub_df = pd.DataFrame()
sub_df['ID'] = test['ID']
sub_df["TARGET"] = lgb.predict(X_test)
sub_df.to_csv("randomForest120.csv", index=False)

In [ ]:
feature_importances = pd.DataFrame(et.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',                                                                 ascending=False)

In [ ]:
feature_importances

In [ ]:
#CAL2250287

In [ ]:
#test = test.reset_index()
sub_df = pd.DataFrame()
sub_df['ID'] = test['ID']
sub_df["TARGET"] = et1.predict(X_test)
sub_df.to_csv("randomForest77.csv", index=False)

In [ ]:

'''from xgboost.sklearn import XGBRegressor

xgb = XGBRegressor()
xgb.fit(X_train, y_train)
predictionsXg = xgb.predict(X_val)
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_val, predictionsXg)'''

In [ ]:
'''test = test.reset_index()
sub_df = pd.DataFrame()
sub_df['ID'] = test['ID']
sub_df["TARGET"] = rf.predict(X_test)
sub_df.to_csv("Rf2508_eolienne_baseline_lgb.csv", index=False)'''